### Guided Project: Analyzing CIA Factbook Data Using SQL

#### INTRODUCTION

In this project, I'll work with data from the [CIA World Factbook](https://www.cia.gov/the-world-factbook/), a compendium of statistics about all of the countries on Earth. The factbook contains demographic information like the following:
- `population` — the global population.
- `population_growth` — the annual population growth rate, as a percentage.
- `area` — the total land and water area.

In this project, I'll use SQL in Jupyter Notebook to analyze data from this database. If you want to work on this project locally, you can download the SQLite factbook.db database [here](https://dsserver-prod-resources-1.s3.amazonaws.com/257/factbook.db)

In [1]:
!conda install -yc conda-forge ipython-sql

Solving environment: ...working... done

# All requested packages already installed.



I'll use the following code to connect our Jupyter Notebook to our database file:

__NB:__ The database file should be in the same folder as the notebook file

In [2]:
pip install emoji --upgrade

Note: you may need to restart the kernel to use updated packages.


In [ ]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

Just to ensure that my connection was correct, I'll write a query to return all tables in the database

In [ ]:
%%sql
SELECT *
FROM sqlite_master
WHERE type='table';


Now lets explore the contents of the facts table as this is the table which contains the data I want to analyse

In [ ]:
%%sql
SELECT *
FROM facts
LIMIT 5;

From this result, I can see teh different columns in the dataset and can probably even deduce what they correspond to. But for clarity, I'll describe each column and what they correspond to.

- `id` - Serial incremental identifier for table
- `code` -Country code
- `name` - Country name
- `area` - Total land and water area in square kilometers
- `area_land` - Total land area in square kilometers
- `area_water` - Total water area in square kilometers
- `population` - Total population of country
- `population_growth` - Annual population growth as a percentage
- `birth_rate` - Annual birth rate, or the number of births per year per 1,000 people.
- `death_rate` - Annual death rate, or the number of deaths per year per 1,000 people.
- `migration_rate` - - Annual migration rate

**Now, I'll write a query to determine the minimum and maximum population and also the fastest and slowest growing population**

In [ ]:
%%sql
SELECT MIN(population) AS 'Minimum Population', 
    MAX(population) AS 'Maximum Population',
    MIN(population_growth) AS 'Minimum Population Growth',
    MAX(population_growth) AS 'Maximum Population Growth'
FROM facts

*Oh wow, who would've thought there was a country with __Zero__ population right? And another country with over 7 billion people yeah? __Hmmmmmmmmmm, Is that not like the population of the whole world???__*

You know what, let's see which countries these are.

In [ ]:
%%sql
SELECT name, population
FROM facts
WHERE population in (SELECT MIN(population) FROM facts)

Oh, It obviously had to be Antarctica. Now let's see the country with the highest population

In [ ]:
%%sql
SELECT name, population
FROM facts
WHERE population in (SELECT MAX(population) FROM facts)

Figures. World is indicated in my dataset as a country.

I'm going to recalculate max and min populations again but this time, I'll exempt the row that contains World. ANd yes, I'll have to exempt world from all my queries from now on

In [ ]:
%%sql
SELECT MIN(population) AS 'Minimum Population', 
    MAX(population) AS 'Maximum Population',
    MIN(population_growth) AS 'Minimum Population Growth',
    MAX(population_growth) AS 'Maximum Population Growth'
FROM facts
WHERE name NOT IN ('World')

Okay, I think its fairer now. 1.3 billion is the highest. I'm guessing that should be China or India. Will probably confirm that later.

For now, I want to calculate the average population of all the countries and also average total surface area

In [ ]:
%%sql
SELECT ROUND(AVG(population), 2) AS 'Average Population', 
    ROUND(AVG(area), 3) AS 'Average Area'
FROM facts
WHERE name NOT IN ('World')

Great. Okay, I'm actually curious.

Let me just confirm the most populous country in the world before moving on, lol.

In [ ]:
%%sql
SELECT name, population
FROM facts
WHERE population in (SELECT MAX(population) FROM facts WHERE name NOT IN ('World'))

I knew it. China. It just had to be. No wonder they are the world manufacturing power.

**_Okay, curiosity satisfied_**

Who else noticed that the average population is quite small. Just over 32 milion. 
I mean, considering the fact that we have the likes of China in this list, tehre has to be an awful lot of countries with really low population to hget this kind of average right. Cos, I mean Nigeria my country even has over 200 million people.

The average area also looks kinda small I think. That's like half of Nigeria's or 60% give or take. I'm really curious to know which countries are this small.

You know what, lets find the countries that have above average values for population and below average values for area

In [ ]:
%%sql
SELECT name, population
FROM facts
WHERE population > (SELECT ROUND(AVG(population), 2) FROM facts WHERE name NOT IN ('World'))
AND name NOT IN ('World')

_Oh wow, they are actually just a handful_

In [ ]:
%%sql
SELECT name, population
FROM facts
WHERE area < (SELECT ROUND(AVG(area), 3) FROM facts WHERE name NOT IN ('World'))
AND name NOT IN ('World')

_Okay, this is a loooooot. And there are reallllly small countries like **Cocos Island**, **Vatican city**, etc_

Okay, let's see which country as the highest population growth rate

In [ ]:
%%sql
SELECT name, population_growth
FROM facts
WHERE population_growth IN (SELECT MAX(population_growth) FROM facts WHERE name NOT IN ('World'))

Okay, this one came as a surprise. Ofcourse I expected it to be in Africa. But for some reason, I did not expect it to be South Sudan.

Next, I want to see which country has more water than land. I've heard there are countries that are mostly water than land.

So first, let's find the ratio of water to land for all the countries yeah


In [ ]:
%%sql
SELECT name, area_land, area_water, ROUND(CAST(area_water AS Float)/CAST(area_land AS Float), 4) AS 'Ratio'
FROM facts
WHERE name != 'World'

From this, the values for the OCeans are empty so it would seem that all the countries here have more land area than water. but let's still confirm that.


In [ ]:
%%sql
SELECT name, area_land, area_water, ROUND(CAST(area_water AS Float)/CAST(area_land AS Float), 4) AS 'Ratio'
FROM facts
WHERE name != 'World'
AND Ratio > 1

Okay, So we have just two countries with a higher water area than land area. `British Indian Ocean Territory` stands out though. Is that ven an Island or a complete water body, lol.	

Now let's explore birth and death rate. let's see which countries have a higher death rate than birth rate cos that would have to be abnormal yeah.

In [ ]:
%%sql
SELECT name, birth_rate, death_rate
FROM facts
WHERE name != 'World'
AND death_rate > birth_rate

_**WHATTTTT???**_

Okay, this is too much. Germany, Japan, Italy???? I'll need to research this to find out why. This should not be.

DFinally, I want to see which countries have the highest population to area ratio. Let's see which countries we can counsider over populated and **probably contract Thanos to snap a finger at them. lmao**


In [ ]:
%%sql
SELECT name, population, area , ROUND(CAST(population AS Float)/area, 2) AS population_area_ratio
FROM facts
WHERE name != 'World'
ORDER BY population_area_ratio DESC

I think this is too much so let me limit the results to the first 30 entries 

In [ ]:
%%sql
SELECT name, population, area , ROUND(CAST(population AS Float)/area, 2) AS population_area_ratio
FROM facts
WHERE name != 'World'
ORDER BY population_area_ratio DESC
LIMIT 30

I really don't care what people say but you see Macau and Monaco, definitely needs a snap of Thanos' fingers. lol.

#### Thank you for following me through this journey and story. Your company is well appreciated.

### THE END